# The Gaussian Distribution

## Table of Contents

- [Background](#background)
- [The Gaussian in one dimension](#the-gaussian-in-one-dimension)
- [The Gaussian in two dimension (aka bivariate normal)](#the-gaussian-in-two-dimension-aka-bivariate-normal)
    - [(1) Uncorrelated bivariate](#1-uncorrelated-bivariate)
    - [(2) Correlated bivariate](#2-correlated-bivariate)
    - [(3) Compact Matrix Form ($\Sigma$-based)](#3-compact-matrix-form--based)
- [Advanced elaboration: Elegance Explored: Exploratory "Derivations"](#elegance-explored-exploratory-derivations)
    - [(I) Compact $\rightarrow$ correlated](#i-compact-correlated)
    - [(II) Compact $\rightarrow$ Uncorrelated](#ii-compact-uncorrelated)

## Background

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

plt.rcParams.update({
    "font.family": "serif",
    "mathtext.fontset": "cm",
    "axes.unicode_minus": False
})

from ipywidgets import (
    FloatSlider, IntSlider, Checkbox,
    HBox, VBox, Layout, interactive_output
)
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401

## The Gaussian in one dimension

In [ ]:
# =============================================================================
# Interactive Gaussian (Normal) Distribution Explorer 
#   FIXES requested:
#   1) Axis fixed: x does NOT re-center on μ; y does NOT auto-rescale with σ/A
#   2) Sliders in 2 rows of 3
#   3) In-plot text: (μ, σ) on one line; (A, f(μ)) on next line
# =============================================================================

# --- LaTeX-ish look (Computer Modern) ---
plt.rcParams.update({
    "font.family": "serif",
    "mathtext.fontset": "cm",
    "axes.unicode_minus": False,
})

# %matplotlib inline


def gaussian_pdf(x, mu, sigma, A=1.0):
    return A * (1.0 / (sigma * np.sqrt(2*np.pi))) * np.exp(-((x - mu)**2) / (2*sigma**2))


# Slider ranges (also used to set fixed y-limit)
MU_MIN, MU_MAX = -5.0, 5.0
SIGMA_MIN, SIGMA_MAX = 0.10, 5.00
A_MIN, A_MAX = 0.10, 2.00

# Fixed y-axis ceiling based on the *maximum possible peak* given slider limits
# peak = A / (sigma * sqrt(2π)) is maximized at (A_max, sigma_min)
Y_MAX_FIXED = 1.10 * (A_MAX / (SIGMA_MIN * np.sqrt(2*np.pi)))


def plot_gaussian(mu=0.0, sigma=1.0, A=1.0, x_span=6.0, n=1500):
    # -----------------------------
    # FIXED X-AXIS:
    #   keep domain symmetric about 0 (not about μ)
    # -----------------------------
    x = np.linspace(-x_span, x_span, int(n))
    y = gaussian_pdf(x, mu, sigma, A)

    fig, ax = plt.subplots(figsize=(10, 5.5))

    # curve + fill + mean line
    ax.plot(x, y, lw=2.8)
    ax.fill_between(x, 0, y, alpha=0.12)
    ax.axvline(mu, ls="--", lw=1.8)

    # styling
    ax.grid(True, alpha=0.20)
    ax.set_xlabel(r"$x$", fontsize=14)
    ax.set_ylabel(r"$f(x)$", fontsize=14)
    ax.set_title(r"Gaussian Distribution", fontsize=18, pad=12)

    # -----------------------------
    # FIXED AXES LIMITS:
    # -----------------------------
    ax.set_xlim(-x_span, x_span)   # fixed relative to 0, only span changes
    ax.set_ylim(0, Y_MAX_FIXED)    # fixed across μ, σ, A

    # formula box
    formula = (
        r"$f(x)=A\cdot \frac{1}{\sigma\sqrt{2\pi}}"
        r"\exp\!\left(-\frac{(x-\mu)^2}{2\sigma^2}\right)$"
    )
    ax.text(
        0.02, 0.96, formula,
        transform=ax.transAxes,
        ha="left", va="top",
        fontsize=18,
        bbox=dict(boxstyle="round,pad=0.35", alpha=0.10)
    )

    # -----------------------------
    # Text readout (requested: A and f(μ) on next line)
    # -----------------------------
    f_mu = gaussian_pdf(mu, mu, sigma, A)
    ax.text(
        0.02, 0.84,
        "\n"rf"$\mu={mu:.3f}$    $\sigma={sigma:.3f}$" "\n"
        rf"$A={A:.3f}$    $f(\mu)={f_mu:.4f}$",
        transform=ax.transAxes,
        ha="left", va="top",
        fontsize=15
    )

    plt.show()


# =============================================================================
# Widgets 2x3
# =============================================================================
w_mu    = FloatSlider(value=0.0, min=MU_MIN,    max=MU_MAX,    step=0.01, description="μ",      continuous_update=True)
w_sigma = FloatSlider(value=0.20, min=SIGMA_MIN, max=SIGMA_MAX, step=0.01, description="σ",      continuous_update=True)
w_A     = FloatSlider(value=2.0, min=A_MIN,     max=A_MAX,     step=0.01, description="A",      continuous_update=True)

w_xspan = FloatSlider(value=3.0, min=2.0, max=12.0, step=0.5, description="x-span", continuous_update=False)
w_n     = IntSlider(  value=1500, min=200, max=4000, step=100, description="n",      continuous_update=False)

# Third slot in row 2: a tiny invisible spacer (keeps 2x3 grid look)
w_blank = FloatSlider(value=0.0, min=0.0, max=0.0, step=1.0, description="", disabled=True)
w_blank.layout.visibility = "hidden"

controls = VBox([
    HBox([w_mu, w_sigma, w_A]),
    HBox([w_xspan, w_n, w_blank]),
])

out = interactive_output(
    plot_gaussian,
    {"mu": w_mu, "sigma": w_sigma, "A": w_A, "x_span": w_xspan, "n": w_n}
)

display(controls, out)


## The Gaussian in two dimension (aka bivariate normal)

### (1) Uncorrelated bivariate

In [ ]:
# =============================================================================
# 2D Gaussian (Multivariate Normal) — UNCORRELATED (ρ fixed to 0)
# -----------------------------------------------------------------------------
# Same layout + plotting as your correlated version, but:
#   - gaussian_2d ignores rho (effectively ρ = 0)
#   - the rho slider is removed
#   - the formula panel is the uncorrelated bivariate normal
#   - the parameter readout no longer shows ρ
# =============================================================================

plt.rcParams.update({
    "font.family": "serif",
    "mathtext.fontset": "cm",   # Computer Modern
    "axes.unicode_minus": False
})

from ipywidgets import (
    FloatSlider, IntSlider, Checkbox,
    HBox, VBox, Layout, interactive_output
)
from mpl_toolkits.mplot3d import Axes3D  # noqa: F401


# =============================================================================
# 2D Gaussian density on a grid (UNCORRELATED: ρ = 0)
# =============================================================================
def gaussian_2d(X, Y, mu_x, mu_y, sigma_x, sigma_y):
    sigma_x = max(float(sigma_x), 1e-9)
    sigma_y = max(float(sigma_y), 1e-9)

    dx = X - float(mu_x)
    dy = Y - float(mu_y)

    Q = (dx**2) / (sigma_x**2) + (dy**2) / (sigma_y**2)
    norm = 1.0 / (2.0 * np.pi * sigma_x * sigma_y)
    Z = norm * np.exp(-0.5 * Q)
    return Z


# =============================================================================
# Plot
# =============================================================================
def plot_gaussian_3d(
    mu_x, mu_y, sigma_x, sigma_y,
    span, grid_n,
    elev, azim,
    show_surface, show_contour
):
    # Grid
    span = max(float(span), 0.5)
    grid_n = int(np.clip(int(grid_n), 60, 260))

    x = np.linspace(-span, span, grid_n)
    y = np.linspace(-span, span, grid_n)
    X, Y = np.meshgrid(x, y)

    Z = gaussian_2d(X, Y, mu_x, mu_y, sigma_x, sigma_y)

    zmax = float(np.max(Z))
    zoff = -0.35 * zmax

    # Figure
    fig = plt.figure(figsize=(10.5, 6.7))
    ax = fig.add_subplot(111, projection="3d")

    # Title (top)
    fig.suptitle("2D Gaussian (Uncorrelated)", fontsize=18, y=0.98)

    # Axes / view
    ax.set_xlabel(r"$x$", fontsize=12, labelpad=10)
    ax.set_ylabel(r"$y$", fontsize=12, labelpad=10)
    ax.set_zlabel(r"$f(x,y)$", fontsize=12, labelpad=10)
    ax.set_xlim(-span, span)
    ax.set_ylim(-span, span)
    ax.set_zlim(zoff, 1.05 * zmax)
    ax.view_init(elev=float(elev), azim=float(azim))
    ax.grid(True, alpha=0.25)

    # Surface (with colour)
    if bool(show_surface):
        ax.plot_surface(
            X, Y, Z,
            cmap="viridis",
            linewidth=0.15,
            antialiased=True,
            rcount=90, ccount=90
        )

    # Contour projection
    if bool(show_contour):
        ax.contourf(
            X, Y, Z,
            zdir="z",
            offset=zoff,
            levels=14,
            cmap="viridis"
        )

    # Formula panel (top-left of the figure, not the 3D axes)
    formula = (
        r"$f(x,y)"
        r"=\frac{1}{2\pi\,\sigma_x\sigma_y}"
        r"\exp\!\left("
        r"-\frac{1}{2}"
        r"\left["
        r"\frac{(x-\mu_x)^2}{\sigma_x^2}"
        r"+\frac{(y-\mu_y)^2}{\sigma_y^2}"
        r"\right]"
        r"\right)$"
    )
    fig.text(
        0.02, 0.90, formula,
        fontsize=18,
        va="top", ha="left",
        bbox=dict(boxstyle="round,pad=0.35", alpha=0.12)
    )

    # Parameter readout (bottom-left)
    fig.text(
        0.02, 0.06,
        rf"$\mu=({mu_x:.2f},{mu_y:.2f}),\ \sigma=({sigma_x:.2f},{sigma_y:.2f})$",
        fontsize=12
    )

    # Give the plot more headroom for the title + formula
    plt.subplots_adjust(top=0.86, left=0.00, right=1.00, bottom=0.00)

    plt.show()


# =============================================================================
# Clean widget layout (tight, top-right-ish block)
# =============================================================================
_w = Layout(width="230px")
_row = Layout(justify_content="flex-start", gap="12px")
_pad = Layout(padding="0px 0px 0px 0px")

w_mx = FloatSlider(value=-0.2, min=-2.0, max=2.0, step=0.05, description="μx", continuous_update=True, layout=_w)
w_my = FloatSlider(value= 0.0, min=-2.0, max=2.0, step=0.05, description="μy", continuous_update=True, layout=_w)
w_sx = FloatSlider(value= 1.0, min=0.25, max=2.5, step=0.05, description="σx", continuous_update=True, layout=_w)
w_sy = FloatSlider(value= 1.1, min=0.25, max=2.5, step=0.05, description="σy", continuous_update=True, layout=_w)

w_span = FloatSlider(value=3.0, min=1.0, max=6.0, step=0.25, description="span", continuous_update=True, layout=_w)
w_gn   = IntSlider(value=120, min=60, max=260, step=10, description="grid", continuous_update=True, layout=_w)

w_el = FloatSlider(value=25.0, min=5.0, max=80.0, step=1.0, description="elev", continuous_update=True, layout=_w)
w_az = FloatSlider(value=-56.0, min=-180.0, max=180.0, step=2.0, description="azim", continuous_update=True, layout=_w)

w_surface = Checkbox(value=True, description="surface", indent=False, layout=Layout(width="120px"))
w_contour = Checkbox(value=True, description="contour", indent=False, layout=Layout(width="120px"))

controls = VBox(
    [
        HBox([w_mx, w_my, w_sx], layout=_row),
        HBox([w_sy, w_span, w_gn], layout=_row),
        HBox([w_el, w_az, w_surface], layout=_row),
        HBox([w_contour], layout=Layout(justify_content="flex-start")),
    ],
    layout=_pad
)

out = interactive_output(
    plot_gaussian_3d,
    {
        "mu_x": w_mx, "mu_y": w_my,
        "sigma_x": w_sx, "sigma_y": w_sy,
        "span": w_span, "grid_n": w_gn,
        "elev": w_el, "azim": w_az,
        "show_surface": w_surface,
        "show_contour": w_contour
    }
)

VBox([controls, out])


### (2) Correlated bivariate

In [ ]:
# =============================================================================
# 2D Gaussian (Multivariate Normal) — Interactive Matplotlib (Clean Layout)
# -----------------------------------------------------------------------------
# What you get:
#   - 3D surface plot + optional contour projection
#   - Formula panel rendered on the figure
#   - Controls grouped tightly under the title (top area)
#   - Tick boxes to toggle: surface / contour
#
# Requirements:
#   pip install ipywidgets
#   (and in Jupyter you may need to enable widgets depending on your setup)
#
# Tip:
#   Use: %matplotlib widget
#   for better 3D interactivity in Jupyter.
# =============================================================================


# =============================================================================
# 2D Gaussian density on a grid
# =============================================================================
def gaussian_2d(X, Y, mu_x, mu_y, sigma_x, sigma_y, rho):
    # Keep Σ positive definite
    rho = float(np.clip(rho, -0.999, 0.999))
    sigma_x = max(float(sigma_x), 1e-9)
    sigma_y = max(float(sigma_y), 1e-9)

    sx2 = sigma_x**2
    sy2 = sigma_y**2
    sxy = rho * sigma_x * sigma_y

    det = sx2 * sy2 - sxy**2
    det = max(float(det), 1e-18)

    inv11 =  sy2 / det
    inv22 =  sx2 / det
    inv12 = -sxy / det

    dx = X - float(mu_x)
    dy = Y - float(mu_y)

    Q = inv11 * dx**2 + 2.0 * inv12 * dx * dy + inv22 * dy**2
    norm = 1.0 / (2.0 * np.pi * np.sqrt(det))
    Z = norm * np.exp(-0.5 * Q)
    return Z


# =============================================================================
# Plot
# =============================================================================
def plot_gaussian_3d(
    mu_x, mu_y, sigma_x, sigma_y, rho,
    span, grid_n,
    elev, azim,
    show_surface, show_contour
):
    # Grid
    span = max(float(span), 0.5)
    grid_n = int(np.clip(int(grid_n), 60, 260))

    x = np.linspace(-span, span, grid_n)
    y = np.linspace(-span, span, grid_n)
    X, Y = np.meshgrid(x, y)

    Z = gaussian_2d(X, Y, mu_x, mu_y, sigma_x, sigma_y, rho)

    zmax = float(np.max(Z))
    zoff = -0.35 * zmax

    # Figure
    fig = plt.figure(figsize=(10.5, 6.7))
    ax = fig.add_subplot(111, projection="3d")

    # Title (top)
    fig.suptitle("2D Gaussian (Multivariate Normal)", fontsize=18, y=0.98)

    # Axes / view
    ax.set_xlabel(r"$x$", fontsize=12, labelpad=10)
    ax.set_ylabel(r"$y$", fontsize=12, labelpad=10)
    ax.set_zlabel(r"$f(x,y)$", fontsize=12, labelpad=10)
    ax.set_xlim(-span, span)
    ax.set_ylim(-span, span)
    ax.set_zlim(zoff, 1.05 * zmax)
    ax.view_init(elev=float(elev), azim=float(azim))
    ax.grid(True, alpha=0.25)

    # Surface (with colour)
    if bool(show_surface):
        ax.plot_surface(
            X, Y, Z,
            cmap="viridis",
            linewidth=0.15,
            antialiased=True,
            rcount=90, ccount=90
        )

    # Contour projection
    if bool(show_contour):
        ax.contourf(
            X, Y, Z,
            zdir="z",
            offset=zoff,
            levels=14,
            cmap="viridis"
        )

    # Formula panel (top-left of the figure, not the 3D axes)
    formula = (
        r"$f(x,y)"
        r"=\frac{1}{2\pi\,\sigma_x\sigma_y\sqrt{1-\rho^2}}"
        r"\exp\!\left("
        r"-\frac{1}{2(1-\rho^2)}"
        r"\left["
        r"\frac{(x-\mu_x)^2}{\sigma_x^2}"
        r"+\frac{(y-\mu_y)^2}{\sigma_y^2}"
        r"-\frac{2\rho(x-\mu_x)(y-\mu_y)}{\sigma_x\sigma_y}"
        r"\right]"
        r"\right)$"
    )
    fig.text(
        0.02, 0.90, formula,
        fontsize=18,
        va="top", ha="left",
        bbox=dict(boxstyle="round,pad=0.35", alpha=0.12)
    )

    # Parameter readout (bottom-left)
    fig.text(
        0.02, 0.06,
        rf"$\mu=({mu_x:.2f},{mu_y:.2f}),\ \sigma=({sigma_x:.2f},{sigma_y:.2f}),\ \rho={rho:.2f}$",
        fontsize=12
    )

    # Give the plot more headroom for the title + formula
    plt.subplots_adjust(top=0.86, left=0.00, right=1.00, bottom=0.00)

    plt.show()


# =============================================================================
# Clean widget layout (tight, top-right-ish block)
# =============================================================================
_w = Layout(width="230px")
_row = Layout(justify_content="flex-start", gap="12px")
_pad = Layout(padding="0px 0px 0px 0px")

w_mx = FloatSlider(value=-0.2, min=-2.0, max=2.0, step=0.05, description="μx", continuous_update=True, layout=_w)
w_my = FloatSlider(value= 0.0, min=-2.0, max=2.0, step=0.05, description="μy", continuous_update=True, layout=_w)
w_sx = FloatSlider(value= 1.0, min=0.25, max=2.5, step=0.05, description="σx", continuous_update=True, layout=_w)
w_sy = FloatSlider(value= 1.1, min=0.25, max=2.5, step=0.05, description="σy", continuous_update=True, layout=_w)
w_rh = FloatSlider(value= 0.0, min=-0.95, max=0.95, step=0.05, description="ρ",  continuous_update=True, layout=_w)

w_span = FloatSlider(value=3.0, min=1.0, max=6.0, step=0.25, description="span", continuous_update=True, layout=_w)
w_gn   = IntSlider(value=120, min=60, max=260, step=10, description="grid", continuous_update=True, layout=_w)

w_el = FloatSlider(value=25.0, min=5.0, max=80.0, step=1.0, description="elev", continuous_update=True, layout=_w)
w_az = FloatSlider(value=-56.0, min=-180.0, max=180.0, step=2.0, description="azim", continuous_update=True, layout=_w)

w_surface = Checkbox(value=True, description="surface", indent=False, layout=Layout(width="120px"))
w_contour = Checkbox(value=True, description="contour", indent=False, layout=Layout(width="120px"))

# TIGHT control block (put under a header, aligned to the right)
controls = VBox(
    [
        HBox([w_mx, w_my, w_sx], layout=_row),
        HBox([w_sy, w_rh, w_span], layout=_row),
        HBox([w_gn, w_el, w_az], layout=_row),
        HBox([w_surface, w_contour], layout=Layout(justify_content="flex-start", gap="20px")),
    ],
    layout=_pad
)

out = interactive_output(
    plot_gaussian_3d,
    {
        "mu_x": w_mx, "mu_y": w_my,
        "sigma_x": w_sx, "sigma_y": w_sy,
        "rho": w_rh,
        "span": w_span, "grid_n": w_gn,
        "elev": w_el, "azim": w_az,
        "show_surface": w_surface,
        "show_contour": w_contour
    }
)

# Place the controls above the plot output (compact, under title area)
VBox([controls, out])


### (3) Compact Matrix Form ($\Sigma$-based)

(The standard general formula you will see in a textbook for the bivariate normal)

The covariance matrix encodes the scale and orientation of the Gaussian through its variances and correlations.

$$
\Sigma =
\begin{pmatrix}
\sigma_x^2 & \rho\,\sigma_x\sigma_y \\
\rho\,\sigma_x\sigma_y & \sigma_y^2
\end{pmatrix},
$$

$$
\Sigma_{xx} = \sigma_x^2,\quad
\Sigma_{yy} = \sigma_y^2,\quad
\Sigma_{xy} = \rho\,\sigma_x\sigma_y
$$

If uncorrelated, then,

$$
\Sigma =
\begin{pmatrix}
\sigma_x^2 & 0 \\
0 & \sigma_y^2
\end{pmatrix}
$$

Final Form:

$$
f(\mathbf{x})
=
\frac{1}{(2\pi)^{d/2}\,|\Sigma|^{1/2}}
\exp\!\left(
-\frac{1}{2}
(\mathbf{x}-\boldsymbol{\mu})^{\top}
\Sigma^{-1}
(\mathbf{x}-\boldsymbol{\mu})
\right)
$$

> **The elegance of the compact form:**<br>
The multivariate Gaussian is fully defined by its mean vector $\mu$ and covariance matrix $\Sigma$; correlation is encoded entirely in the off-diagonal terms of $\Sigma$ (off-diagonal term(s) being $\Sigma_{xy} = \rho\,\sigma_x\sigma_y$). 

This makes more sense when u now try to explain why the off-diagonal code is 0 in uncorrelated. (feels obviously simple now right?)


In [ ]:
# =============================================================================
# 2D Gaussian — COMPACT MATRIX FORM (Σ-based)
# -----------------------------------------------------------------------------
# This version:
#   - Implements the Gaussian using the matrix definition
#   - Explicitly constructs Σ and Σ^{-1}
#   - Displays the compact theoretical formula
#   - Is equivalent to the correlated scalar-expanded version
# =============================================================================

# =============================================================================
# 2D Gaussian via matrix form
# =============================================================================
def gaussian_2d_matrix(X, Y, mu_x, mu_y, sigma_x, sigma_y, rho):
    # Mean vector
    mu = np.array([mu_x, mu_y])

    # Covariance matrix
    sigma_x = max(float(sigma_x), 1e-9)
    sigma_y = max(float(sigma_y), 1e-9)
    rho = float(np.clip(rho, -0.999, 0.999))

    Sigma = np.array([
        [sigma_x**2, rho * sigma_x * sigma_y],
        [rho * sigma_x * sigma_y, sigma_y**2]
    ])

    Sigma_inv = np.linalg.inv(Sigma)
    det_Sigma = np.linalg.det(Sigma)

    # Stack grid into vectors
    pos = np.dstack((X, Y))
    diff = pos - mu

    # Quadratic form: (x−μ)^T Σ^{-1} (x−μ)
    Q = (
        Sigma_inv[0, 0] * diff[..., 0]**2
        + 2 * Sigma_inv[0, 1] * diff[..., 0] * diff[..., 1]
        + Sigma_inv[1, 1] * diff[..., 1]**2
    )

    norm = 1.0 / (2.0 * np.pi * np.sqrt(det_Sigma))
    Z = norm * np.exp(-0.5 * Q)
    return Z


# =============================================================================
# Plot
# =============================================================================
def plot_gaussian_3d(
    mu_x, mu_y, sigma_x, sigma_y, rho,
    span, grid_n,
    elev, azim,
    show_surface, show_contour
):
    span = max(float(span), 0.5)
    grid_n = int(np.clip(int(grid_n), 60, 260))

    x = np.linspace(-span, span, grid_n)
    y = np.linspace(-span, span, grid_n)
    X, Y = np.meshgrid(x, y)

    Z = gaussian_2d_matrix(X, Y, mu_x, mu_y, sigma_x, sigma_y, rho)

    zmax = float(np.max(Z))
    zoff = -0.35 * zmax

    fig = plt.figure(figsize=(10.5, 6.7))
    ax = fig.add_subplot(111, projection="3d")

    fig.suptitle("2D Gaussian (Matrix Form)", fontsize=18, y=0.98)

    ax.set_xlabel(r"$x$")
    ax.set_ylabel(r"$y$")
    ax.set_zlabel(r"$f(x,y)$")
    ax.set_xlim(-span, span)
    ax.set_ylim(-span, span)
    ax.set_zlim(zoff, 1.05 * zmax)
    ax.view_init(elev=float(elev), azim=float(azim))
    ax.grid(True, alpha=0.25)

    if show_surface:
        ax.plot_surface(
            X, Y, Z,
            cmap="viridis",
            linewidth=0.15,
            antialiased=True
        )

    if show_contour:
        ax.contourf(
            X, Y, Z,
            zdir="z",
            offset=zoff,
            levels=14,
            cmap="viridis"
        )

    # Compact matrix formula
    formula = (
        r"$f(\mathbf{x})"
        r"=\frac{1}{(2\pi)|\Sigma|^{1/2}}"
        r"\exp\!\left("
        r"-\frac{1}{2}"
        r"(\mathbf{x}-\boldsymbol{\mu})^\top"
        r"\Sigma^{-1}"
        r"(\mathbf{x}-\boldsymbol{\mu})"
        r"\right)$"
    )

    fig.text(
        0.02, 0.90, formula,
        fontsize=18,
        va="top", ha="left",
        bbox=dict(boxstyle="round,pad=0.35", alpha=0.12)
    )

    fig.text(
    0.02, 0.06,
    rf"$\boldsymbol{{\mu}}=({mu_x:.2f},{mu_y:.2f}),\ "
    rf"\Sigma_{{xx}}=\sigma_x^2,\ \Sigma_{{yy}}=\sigma_y^2,\ "
    rf"\Sigma_{{xy}}=\rho\,\sigma_x\sigma_y$",
    fontsize=12
    )



    plt.subplots_adjust(top=0.86, left=0.00, right=1.00, bottom=0.00)
    plt.show()


# =============================================================================
# Widgets
# =============================================================================
_w = Layout(width="230px")
_row = Layout(justify_content="flex-start", gap="12px")

w_mx = FloatSlider(value=0.0, min=-2, max=2, step=0.05, description="μx", layout=_w)
w_my = FloatSlider(value=0.0, min=-2, max=2, step=0.05, description="μy", layout=_w)
w_sx = FloatSlider(value=1.0, min=0.25, max=2.5, step=0.05, description="σx", layout=_w)
w_sy = FloatSlider(value=1.0, min=0.25, max=2.5, step=0.05, description="σy", layout=_w)
w_rh = FloatSlider(value=0.0, min=-0.95, max=0.95, step=0.05, description="ρ", layout=_w)

w_span = FloatSlider(value=3.0, min=1, max=6, step=0.25, description="span", layout=_w)
w_gn = IntSlider(value=120, min=60, max=260, step=10, description="grid", layout=_w)

w_el = FloatSlider(value=25, min=5, max=80, step=1, description="elev", layout=_w)
w_az = FloatSlider(value=-55, min=-180, max=180, step=2, description="azim", layout=_w)

w_surface = Checkbox(value=True, description="surface")
w_contour = Checkbox(value=True, description="contour")

controls = VBox([
    HBox([w_mx, w_my, w_sx], layout=_row),
    HBox([w_sy, w_rh, w_span], layout=_row),
    HBox([w_gn, w_el, w_az], layout=_row),
    HBox([w_surface, w_contour], layout=_row),
])

out = interactive_output(
    plot_gaussian_3d,
    dict(
        mu_x=w_mx, mu_y=w_my,
        sigma_x=w_sx, sigma_y=w_sy,
        rho=w_rh,
        span=w_span, grid_n=w_gn,
        elev=w_el, azim=w_az,
        show_surface=w_surface,
        show_contour=w_contour
    )
)

VBox([controls, out])


----
----

## Elegance Explored: Exploratory "Derivations"

The compact matrix form of the Gaussian is flexible: it represents both correlated and uncorrelated cases, depending entirely on the covariance matrix

$$
\text{Same formula}
\;+\;
\text{different } \Sigma
\;\Longrightarrow\;
\text{different geometry}
$$

or to more formally say the exact same thing (what u may see in some textbooks);

$$
f(\mathbf{x};\Sigma)\quad\text{with varying } \Sigma
\;\Longrightarrow\;
\text{distinct Gaussian geometries}
$$

The compact matrix form defines the general multivariate Gaussian; correlation emerges only through the choice of the covariance matrix.

----

### (I) Compact $\rightarrow$ correlated 

Start from the multivariate normal density in compact (matrix) form for the two-dimensional case:

$$
f(\mathbf{x})
=
\frac{1}{(2\pi)^{1}\,|\Sigma|^{1/2}}
\exp\!\left(
-\frac{1}{2}
(\mathbf{x}-\boldsymbol{\mu})^{\top}
\Sigma^{-1}
(\mathbf{x}-\boldsymbol{\mu})
\right),
$$

with

$$
\mathbf{x}=
\begin{pmatrix}
x\\
y
\end{pmatrix},
\qquad
\boldsymbol{\mu}=
\begin{pmatrix}
\mu_x\\
\mu_y
\end{pmatrix}.
$$

#### 1) Choose the covariance matrix with correlation $\rho$

For a correlated bivariate Gaussian, the covariance matrix is

$$
\Sigma=
\begin{pmatrix}
\sigma_x^2 & \rho\,\sigma_x\sigma_y \\
\rho\,\sigma_x\sigma_y & \sigma_y^2
\end{pmatrix}.
$$

#### 2) Compute the determinant of $\Sigma$

For a symmetric $2\times 2$ matrix of the form

$$
\begin{pmatrix}
a & b\\
b & c
\end{pmatrix},
$$

the determinant is $ac-b^2$.

Applying this here gives

$$
|\Sigma|
=
\sigma_x^2\sigma_y^2-(\rho\sigma_x\sigma_y)^2
=
\sigma_x^2\sigma_y^2(1-\rho^2).
$$

Therefore,

$$
|\Sigma|^{1/2}
=
\sigma_x\sigma_y\sqrt{1-\rho^2}.
$$

#### 3) Compute the inverse of $\Sigma$

The inverse of a symmetric $2\times 2$ matrix is

$$
\begin{pmatrix}
a & b\\
b & c
\end{pmatrix}^{-1}
=
\frac{1}{ac-b^2}
\begin{pmatrix}
c & -b\\
-b & a
\end{pmatrix}.
$$

Applying this to $\Sigma$ yields

$$
\Sigma^{-1}
=
\frac{1}{\sigma_x^2\sigma_y^2(1-\rho^2)}
\begin{pmatrix}
\sigma_y^2 & -\rho\sigma_x\sigma_y\\
-\rho\sigma_x\sigma_y & \sigma_x^2
\end{pmatrix}.
$$

#### 4) Expand the quadratic form

Define the displacement vector

$$
\mathbf{r}
=
\mathbf{x}-\boldsymbol{\mu}
=
\begin{pmatrix}
x-\mu_x\\
y-\mu_y
\end{pmatrix}.
$$

Then the quadratic form becomes

$$
\mathbf{r}^{\top}\Sigma^{-1}\mathbf{r}
=
\frac{1}{\sigma_x^2\sigma_y^2(1-\rho^2)}
\Big[
\sigma_y^2(x-\mu_x)^2
-2\rho\sigma_x\sigma_y(x-\mu_x)(y-\mu_y)
+\sigma_x^2(y-\mu_y)^2
\Big].
$$

Dividing through term by term gives

$$
\mathbf{r}^{\top}\Sigma^{-1}\mathbf{r}
=
\frac{1}{1-\rho^2}
\left[
\frac{(x-\mu_x)^2}{\sigma_x^2}
+
\frac{(y-\mu_y)^2}{\sigma_y^2}
-
\frac{2\rho(x-\mu_x)(y-\mu_y)}{\sigma_x\sigma_y}
\right].
$$

#### 5) Substitute back into the compact density

Substituting the determinant and expanded quadratic form into the compact expression gives

$$
f(x,y)
=
\frac{1}{2\pi\,\sigma_x\sigma_y\sqrt{1-\rho^2}}
\exp\!\left(
-\frac{1}{2(1-\rho^2)}
\left[
\frac{(x-\mu_x)^2}{\sigma_x^2}
+
\frac{(y-\mu_y)^2}{\sigma_y^2}
-
\frac{2\rho(x-\mu_x)(y-\mu_y)}{\sigma_x\sigma_y}
\right]
\right).
$$

This is exactly the <b>correlated (bivariate) Gaussian</b> written in scalar-expanded form, derived directly from the compact matrix definition.


----

### (II) Compact $\rightarrow$ Uncorrelated 

Start from the multivariate normal density (compact form) for the two-dimensional case:

$$
f(\mathbf{x})
=
\frac{1}{(2\pi)^{1}\,|\Sigma|^{1/2}}
\exp\!\left(
-\frac{1}{2}
(\mathbf{x}-\boldsymbol{\mu})^{\top}
\Sigma^{-1}
(\mathbf{x}-\boldsymbol{\mu})
\right),
$$

with

$$
\mathbf{x}=
\begin{pmatrix}
x\\
y
\end{pmatrix},
\qquad
\boldsymbol{\mu}=
\begin{pmatrix}
\mu_x\\
\mu_y
\end{pmatrix}.
$$

#### 1) Choose an uncorrelated covariance matrix

Uncorrelated means the off-diagonal covariance terms are zero, so $\Sigma$ is diagonal:

$$
\Sigma=
\begin{pmatrix}
\sigma_x^2 & 0 \\
0 & \sigma_y^2
\end{pmatrix}.
$$

#### 2) Compute the determinant of $\Sigma$

For a diagonal matrix, the determinant is the product of diagonal entries:

$$
|\Sigma|
=
\sigma_x^2 \sigma_y^2.
$$

Therefore,

$$
|\Sigma|^{1/2}
=
\sigma_x\sigma_y.
$$

#### 3) Compute the inverse of $\Sigma$

The inverse of a diagonal matrix is obtained by inverting each diagonal entry:

$$
\Sigma^{-1}
=
\begin{pmatrix}
\frac{1}{\sigma_x^2} & 0 \\
0 & \frac{1}{\sigma_y^2}
\end{pmatrix}.
$$

#### 4) Expand the quadratic form

Define the displacement vector:

$$
\mathbf{r}
=
\mathbf{x}-\boldsymbol{\mu}
=
\begin{pmatrix}
x-\mu_x\\
y-\mu_y
\end{pmatrix}.
$$

Compute the quadratic form:

$$
\mathbf{r}^{\top}\Sigma^{-1}\mathbf{r}
=
\begin{pmatrix}
x-\mu_x & y-\mu_y
\end{pmatrix}
\begin{pmatrix}
\frac{1}{\sigma_x^2} & 0 \\
0 & \frac{1}{\sigma_y^2}
\end{pmatrix}
\begin{pmatrix}
x-\mu_x\\
y-\mu_y
\end{pmatrix}.
$$

Because $\Sigma^{-1}$ is diagonal, cross-terms vanish and this reduces to

$$
\mathbf{r}^{\top}\Sigma^{-1}\mathbf{r}
=
\frac{(x-\mu_x)^2}{\sigma_x^2}
+
\frac{(y-\mu_y)^2}{\sigma_y^2}.
$$

#### 5) Substitute back into the compact density

Substitute $|\Sigma|^{1/2}=\sigma_x\sigma_y$ and the expanded quadratic form:

$$
f(x,y)
=
\frac{1}{2\pi\,\sigma_x\sigma_y}
\exp\!\left(
-\frac{1}{2}
\left[
\frac{(x-\mu_x)^2}{\sigma_x^2}
+
\frac{(y-\mu_y)^2}{\sigma_y^2}
\right]
\right).
$$

This is exactly the **uncorrelated (axis-aligned) bivariate Gaussian** in scalar-expanded form, derived directly from the compact matrix definition.


----
----
----

In [ ]:
import sys
from pathlib import Path
import importlib

# -----------------------------------------------------------------------------
# Robustly locate: <repo>/notebooks/For_Author
# -----------------------------------------------------------------------------
def _find_for_author_dir() -> Path:
    for base in [Path.cwd(), *Path.cwd().parents]:
        candidate = base / "notebooks" / "For_Author"
        if candidate.is_dir():
            return candidate
    raise FileNotFoundError("Could not find notebooks/For_Author by walking up from cwd.")

author_tools = _find_for_author_dir()

# Put it FIRST on sys.path (important if there are name collisions)
sys.path.insert(0, str(author_tools))

# Fresh import (handles notebook re-runs cleanly)
mod = importlib.import_module("retrieve_headings")
importlib.reload(mod)

make_toc = mod.make_toc

make_toc("Gaussian_Distribution.ipynb")